Below is an example of the sift algorithm, with the trilinear interpolation smoothing of a histogram tensor of the generate descriptors being implemented in hardware using our overlay, histogram_tensor

This overlay stream in inputs for the histogram tensor calculations, which are done on the board, and the resulting structure is stream back out and placed back into the software structure for further calculations

In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, findScaleSpaceExtrema, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2



img1 = cv2.imread('dino_data/dino02.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('dino_data/dino05.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
num_octaves = computeNumberOfOctaves(base_image.shape)
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
dog_images = generateDoGImages(gaussian_images)
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
keypoints = removeDuplicateKeypoints(keypoints)
kp1 = convertKeypointsToInputImageSize(keypoints)
des1 = generateDescriptors_hardware(keypoints, gaussian_images)

base_image = generateBaseImage(img2, sigma, assumed_blur)
num_octaves = computeNumberOfOctaves(base_image.shape)
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
dog_images = generateDoGImages(gaussian_images)
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
keypoints = removeDuplicateKeypoints(keypoints)
kp2 = convertKeypointsToInputImageSize(keypoints)
des2 = generateDescriptors_hardware(keypoints, gaussian_images)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)


# Estimate homography between template and scene
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

# Draw detected template in scene image
h, w = img1.shape
pts = np.float32([[0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, M)

img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

h1, w1 = img1.shape
h2, w2 = img2.shape
nWidth = w1 + w2
nHeight = max(h1, h2)
hdif = int((h2 - h1) / 2)
newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

for i in range(3):
    newimg[hdif:hdif + h1, :w1, i] = img1
    newimg[:h2, w1:w1 + w2, i] = img2

# Draw SIFT keypoint matches
for m in good:
    pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
    pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
    cv2.line(newimg, pt1, pt2, (255, 0, 0))

plt.imshow(newimg)
plt.show()